## SOURCE TO BRONZE LAYER

### Process:

> The function fetches daily stock data using the Yahoo Finance API (`yfinance`).


In [ ]:
# Import necessary libraries
import pandas as pd
from datetime import timedelta
import yfinance as yf
from requests import Session
from pyrate_limiter import Limiter, Duration, RequestRate
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from PortfolioTracker import DateTimeUtil, GlobalPath

In [ ]:
# Importing Common Utility Funcation
%run ../COMMON/common_utility.ipynb

In [ ]:
# Define Constants file paths
HOLDINGS_HISTORY_PATH = GlobalPath(
    "DATA/SOURCE/Holdings/HoldingsHistory_data.csv"
)
STOCKDATA_BRONZE_LAYER_PATH = GlobalPath("DATA/BRONZE/StockData")

In [ ]:
# Setting cache location for yfinance
yf.set_tz_cache_location(".cache")


# Rate limiting setup
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass


history_rate = RequestRate(1, Duration.SECOND * 2)
limiter = Limiter(history_rate)
session = CachedLimiterSession(
    limiter=limiter,
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache(".cache/session", expire_after=timedelta(hours=1)),
)

In [ ]:
def process_file(stock_ticker, date, output_file):
    """Fetch historical stock data and save it to a CSV file."""
    df = stock_ticker.history(
        start=date.start_date,
        end=min(date.end_date, DateTimeUtil.today()),
        interval="1d",
        actions=True,
        rounding=True,
    )
    if df.empty:
        raise Exception(
            f"No data fetched for : from {date.start_date} to {date.end_date}"
        )
    df = df.reset_index()
    df = replace_punctuation_from_columns(df)
    df = fix_duplicate_column_names(df)
    df = df.round(2)
    df.to_csv(output_file, index=False)
    print(f"Data processed and saved to: {output_file}")

In [ ]:
# Dictionary for stock ticker overrides
OVERWRITE_TICKERS = {
    "AWL": "AWL.NS",
    "BANKBARODA": "BANKBARODA.NS",
    "BCG": "BCG.NS",
    "CAMPUS": "CAMPUS.NS",
    "CRESSAN": "CRESSAN.BO",
    "CTL": "CTL.BO",
    "DATAPATTNS": "DATAPATTNS.NS",
    "DEEPAKNTR": "DEEPAKNTR.NS",
    "FCSSOFT": "FCSSOFT.NS",
    "GITARENEW": "GITARENEW.BO",
    "GRAVITA": "GRAVITA.NS",
    "GREENPOWER": "GREENPOWER.NS",
    "GREENPOWER_RE": "GREENPOWER.NS",
    "HFCL": "HFCL.NS",
    "IDFCFIRSTB": "IDFCFIRSTB.NS",
    "JPPOWER": "JPPOWER.NS",
    "KNRCON": "KNRCON.NS",
    "KPEL": "KPEL.BO",
    "KPITTECH": "KPITTECH.NS",
    "LLOYDSENGG": "LLOYDSENGG.NS",
    "PNB": "PNB.NS",
    "PRAKASHSTL": "PRAKASHSTL.NS",
    "RAGHUSYN": "RAGHUSYN.BO",
    "RPOWER": "RPOWER.NS",
    "RTNINDIA": "RTNINDIA.NS",
    "RTNPOWER": "RTNPOWER.NS",
    "SINTEX": "SINTEX.NS",
    "SUZLON": "SUZLON.NS",
    "SWORDEDGE": "SWORDEDGE.BO",
    "TATAMOTORS": "TATAMOTORS.NS",
    "TATAPOWER": "TATAPOWER.NS",
    "TRIDENT": "TRIDENT.NS",
    "TTML": "TTML.NS",
    "URJA": "URJA.NS",
    "VIJIFIN": "VIJIFIN.NS",
    "VIKASLIFE": "VIKASLIFE.NS",
    "VIRTUALG": "VIRTUALG.BO",
    "VISESHINFO": "VISESHINFO.NS",
    "WIPRO": "WIPRO.NS",
    "YAMNINV": "YAMNINV.BO",
    "ZOMATO": "ZOMATO.NS",
    "BAJAJHFL": "BAJAJHFL.NS",
    "BHAGERIA": "BHAGERIA.NS",
    "BPCL": "BPCL.NS",
    "GOLDBEES": "GOLDBEES.NS",
    "HERANBA": "HERANBA.NS",
    "IDEA": "IDEA.NS",
    "INFY": "INFY.NS",
    "IRCTC": "IRCTC.NS",
    "LICI": "LICI.NS",
    "MIRAE_ASSET_ELSS_TAX_SAVER_FUND_DIRECT_PLAN_GROWTH": "0P00017844.BO",
    "NIFTYBEES": "NIFTYBEES.NS",
    "SBIN": "SBIN.NS",
    "SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH": "0P0000XVL9.BO",
    "TATACHEM": "TATACHEM.NS",
    "VOLTAS": "VOLTAS.NS",
    "YESBANK": "YESBANK.NS",
}

In [ ]:
# Load holdings data from CSV
df_holdings_history = pd.read_csv(HOLDINGS_HISTORY_PATH)
df_holdings_history["min_date"] = pd.to_datetime(
    df_holdings_history["min_date"]
)
df_holdings_history["max_date"] = pd.to_datetime(
    df_holdings_history["max_date"]
)
print(f"Loaded data from: {HOLDINGS_HISTORY_PATH}")

In [ ]:
# Fetch stock data and process it.
for _, row in df_holdings_history.iterrows():
    print(f"\nProcessing data for symbol {row['symbol']}:")
    try:
        stock_ticker = yf.Ticker(
            OVERWRITE_TICKERS.get(row["symbol"], row["isin"]),
            session=session,
        )
        date_list = generate_date_list(row["min_date"].to_pydatetime(), row["max_date"].to_pydatetime())
        for date in date_list:
            output_file = STOCKDATA_BRONZE_LAYER_PATH.joinpath(f"{row['symbol']}_{date.year:04d}_{date.month:02d}.csv")
            if output_file.exists() and date.month_difference(DateTimeUtil.today()) >= 1:
                continue
            process_file(stock_ticker, date, output_file)
    except Exception as e:
        print(f"Error processing {row['symbol']}:\n{e}")